In [10]:
import numpy as np
import pandas as pd
from tabulate import tabulate
# Loan with varying interest rates and 5% decreasing payments
# Timeline: t = 0,...,27


In [2]:
# Interest rates by year:
# - 2.5% for the first 5 years
# - 3.5% for the next 5 years
# - 4% for the last 17 years
i = np.array([0.025]*5 + [0.035]*5 + [0.04]*17)  # length = 27

# Discount factors v_t
# v_0 = 1
# v_t = product over j=1..t of 1/(1+i_j)
discount_factor = np.concatenate(([1.0], np.cumprod(1/(1+i))))


In [3]:
# Cash inflows:
# - 30,000 EUR at t=0
# - 10,000 EUR at t=1
# - 15,000 EUR at t=2
PV = 30000 * discount_factor[0] + 10000 * discount_factor[1] + 15000 * discount_factor[2]


In [4]:
# 25 yearly payments from t=3 to t=27
# Each year payment decreases by 5%

pay_ratio = (0.95) ** np.arange(25)   # A, A*0.95, A*0.95^2, ...

# S = weighted sum of discount factors
S = np.sum(pay_ratio * discount_factor[3:28])

# Solve for first payment A
A = PV / S


In [5]:
years = np.arange(3, 28)        # payment years
K = -A * pay_ratio              # negative = outflow for Cynthia

payments_df = pd.DataFrame({
    "Year": years,
    "Payment": K,
    "DiscountFactor": discount_factor[3:28],
    "PV Contribution": (-K) * discount_factor[3:28]
})


In [6]:
pv_payments = np.sum((-K) * discount_factor[3:28])

check = pd.Series({
    "PV_disbursements": PV,
    "PV_payments": pv_payments,
    "Difference": pv_payments - PV
})


In [12]:
print(f"First payment A at t=3: {A:,.2f} EUR")
print(f"Total PV of disbursements: {PV:,.2f} EUR")
print(f"Total PV of payments:      {pv_payments:,.2f} EUR")
print(f"Difference:                {pv_payments - PV:.6f} EUR")

print("\nPayment Schedule:")
print(tabulate(payments_df_rounded, headers="keys", tablefmt="compact", showindex=False))



First payment A at t=3: 5,402.26 EUR
Total PV of disbursements: 54,033.31 EUR
Total PV of payments:      54,033.31 EUR
Difference:                -0.000000 EUR

Payment Schedule:
  Year    Payment    DiscountFactor    PV Contribution
------  ---------  ----------------  -----------------
     3   -5402.26              0.93            5016.54
     4   -5132.15              0.91            4649.47
     5   -4875.54              0.88            4309.27
     6   -4631.76              0.85            3955.37
     7   -4400.18              0.83            3630.53
     8   -4180.17              0.8             3332.37
     9   -3971.16              0.77            3058.7
    10   -3772.6               0.74            2807.5
    11   -3583.97              0.72            2564.54
    12   -3404.77              0.69            2342.61
    13   -3234.53              0.66            2139.89
    14   -3072.81              0.64            1954.7
    15   -2919.17              0.61            1785.55